# RAG with Postgres flexible server

You can run this notebook after running succesfully the "RAG - Postgresql - create embeddings" notebook. 

In [ ]:
from dotenv import load_dotenv
import pandas as pd
from IPython.display import display, HTML, JSON, Markdown
import os

# Configure environment variables
load_dotenv()

# Configure OpenAI API
OPENAI_GPT35_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT35_DEPLOYMENT_NAME")
OPENAI_GPT4_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT4_DEPLOYMENT_NAME")
OPENAI_GPT4V_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT4V_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_DALLE_DEPLOYMENT_NAME = os.getenv("OPENAI_DALLE_DEPLOYMENT_NAME")

OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# configure postgres
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_HOST = os.getenv("POSTGRES_HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [ ]:
embeddingmodel = AzureOpenAIEmbeddings(
    deployment=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME,
    model=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME,
    azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
    chunk_size = 1)

In [ ]:
#we use the tenacity library to create delays and retries when calling openAI embeddings to avoid hitting throttling limits
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def calc_embeddings(text):
    deployment = OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME
    # replace newlines, which can negatively affect performance.
    txt = text.replace("\n", " ")
    return embeddingmodel.embed_query(txt)

In [ ]:
# connect to postgres
import psycopg2

sslmode = "require"
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(POSTGRES_HOST, POSTGRES_USER, POSTGRES_DB, POSTGRES_PASSWORD, sslmode)
conn = psycopg2.connect(conn_string) 
print("Connection established")
cursor = conn.cursor()

In [ ]:
# Checking the vector extension exists
show_extensions_query = "SHOW azure.extensions;"
cursor.execute(show_extensions_query)
conn.commit()
results = cursor.fetchall()
for row in results:
    print(row)

In [ ]:
from pgvector.psycopg2 import register_vector
from psycopg2 import Error
from psycopg2 import sql
import numpy as np

# Register 'pgvector' type for the 'embedding' column
register_vector(conn)
table_name = "embeddings"

# perform a similarity search between a query and the ingested documents
question = "Why does the coffin prepared for Queequeg become Ishmael's life buoy once the Pequod sinks?"
retrieve_k = 1 # for retrieving the top k reviews from the database
# Generate embeddings for the question and retrieve the top k document chunks
questionEmbedding = calc_embeddings(question)


select_query = f"SELECT content FROM {table_name} ORDER BY embedding <-> %s LIMIT {retrieve_k}"
cursor = conn.cursor()
cursor.execute(select_query, (np.array(questionEmbedding),))
results = cursor.fetchall()

answer = results[0][0]
print(answer)

In [ ]:
from openai import AzureOpenAI
clientOpenAI = AzureOpenAI(
  azure_endpoint = OPENAI_DEPLOYMENT_ENDPOINT, 
  api_key=OPENAI_API_KEY,  
  api_version="2023-05-15"
)

def call_openAI(text):
    response = clientOpenAI.chat.completions.create(
        model=OPENAI_GPT35_DEPLOYMENT_NAME,
        messages = text,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    return response.choices[0].message.content

In [ ]:
prompt = 'Question: {}'.format(question) + '\n' + 'Information: {}'.format(answer)
# prepare prompt
messages = [{"role": "system", "content": "You are a HELPFUL assistant answering users questions. Answer the question using the provided information and do not add anything else."},
            {"role": "user", "content": prompt}]

result = call_openAI(messages)
display(HTML(result))